In [31]:
import random

# Definimos las dimensiones de la sala
TAMAÑO_SALA = 5

# Mapa inicial con ingenieros en algunas posiciones (I = Ingeniero, - = Celda vacía)
MAPA = [["-" for _ in range(TAMAÑO_SALA)] for _ in range(TAMAÑO_SALA)]
MAPA[2][2] = "I"
MAPA[4][1] = "I"
MAPA[0][3] = "I"

# Conjunto de terminales y funciones
TERMINALES = ["N", "S", "E", "O", "Detectar", "Entregar"]
FUNCIONES = ["IF", "SECUENCIA"]

# Función para mover el robot
def mover(robot, direccion):
    if direccion == "N" and robot["y"] > 0:
        robot["y"] -= 1
    elif direccion == "S" and robot["y"] < TAMAÑO_SALA - 1:
        robot["y"] += 1
    elif direccion == "E" and robot["x"] < TAMAÑO_SALA - 1:
        robot["x"] += 1
    elif direccion == "O" and robot["x"] > 0:
        robot["x"] -= 1
    robot["movimientos"] += 1

# Función de aptitud
def calcular_aptitud(mapa, programa):
    robot = {"x": 0, "y": 0, "galletas_entregadas": 0, "movimientos": 0, "ingenieros_detectados": 0}

    def ejecutar_instruccion(instr):
        if instr in ["N", "S", "E", "O"]:
            mover(robot, instr)
        elif instr == "Detectar":
            if mapa[robot["x"]][robot["y"]] == "I":
                robot["ingenieros_detectados"] += 1
                return True
            return False
        elif instr == "Entregar":
            if mapa[robot["x"]][robot["y"]] == "I":
                robot["galletas_entregadas"] += 1
                mapa[robot["x"]][robot["y"]] = "-"
        return None

    def ejecutar_programa(prog):
        if isinstance(prog, str):
            ejecutar_instruccion(prog)
        elif prog[0] == "SECUENCIA":
            for accion in prog[1:]:
                ejecutar_programa(accion)
        elif prog[0] == "IF":
            if len(prog) != 4:
                return
            if ejecutar_instruccion(prog[1]):
                ejecutar_programa(prog[2])
            else:
                ejecutar_programa(prog[3])

    ejecutar_programa(programa)
    return robot

# Generar programas aleatorios
def generar_programa(profundidad):
    if profundidad == 0:
        return random.choice(TERMINALES)
    funcion = random.choice(FUNCIONES)
    if funcion == "SECUENCIA":
        return ["SECUENCIA"] + [generar_programa(profundidad - 1) for _ in range(random.randint(2, 4))]
    elif funcion == "IF":
        return ["IF", "Detectar", generar_programa(profundidad - 1), generar_programa(profundidad - 1)]

# Cruce genético
def cruce(p1, p2):
    if not isinstance(p1, list) or not isinstance(p2, list):
        return p1, p2
    punto1 = random.randint(1, len(p1) - 1)
    punto2 = random.randint(1, len(p2) - 1)
    hijo1 = p1[:punto1] + p2[punto2:]
    hijo2 = p2[:punto2] + p1[punto1:]
    return hijo1, hijo2

# Mutación
def mutar(programa):
    if not isinstance(programa, list):
        return random.choice(TERMINALES)
    if random.random() < 0.30:
        return generar_programa(2)
    return [programa[0]] + [mutar(sub) for sub in programa[1:]]

# Inicializar población
def inicializar_poblacion(tamaño, profundidad):
    return [generar_programa(profundidad) for _ in range(tamaño)]

# Evolución
def evolucionar(mapa, poblacion):
    aptitudes = []
    for programa in poblacion:
        resultado = calcular_aptitud([fila[:] for fila in mapa], programa)
        aptitud = resultado["galletas_entregadas"] * 100 - resultado["movimientos"]
        aptitudes.append((programa, aptitud, resultado))

    aptitudes.sort(key=lambda x: -x[1])
    nueva_poblacion = [aptitudes[0][0], aptitudes[1][0]]

    while len(nueva_poblacion) < len(poblacion):
        p1, _, _ = random.choice(aptitudes[:10])
        p2, _, _ = random.choice(aptitudes[:10])
        hijo1, hijo2 = cruce(p1, p2)
        nueva_poblacion += [mutar(hijo1), mutar(hijo2)]

    return nueva_poblacion, aptitudes[0][2]

# Main
def main():
    poblacion = inicializar_poblacion(20, 3)
    generaciones = 150
    mejor_resultado = None

    for _ in range(generaciones):
        poblacion, mejor_resultado = evolucionar(MAPA, poblacion)

    mejor_programa = poblacion[0]

    print("\n--- Mejor Programa Encontrado ---")
    imprimir_programa(mejor_programa)

    print(f"\nAptitud final obtenida: {mejor_resultado['galletas_entregadas'] * 100 - mejor_resultado['movimientos']}")
    print(f"Galletas entregadas: {mejor_resultado['galletas_entregadas']}")
    print(f"Ingenieros detectados: {mejor_resultado['ingenieros_detectados']}")

def imprimir_programa(prog, nivel=0):
    indent = "    " * nivel
    if isinstance(prog, str):
        print(f"{indent}- {prog}")
    elif prog[0] == "SECUENCIA":
        print(f"{indent}SECUENCIA (Ejecuta en orden):")
        for accion in prog[1:]:
            imprimir_programa(accion, nivel+1)
    elif prog[0] == "IF":
        print(f"{indent}IF (Condicional):")
        print(f"{indent}    Si Detectar es True:")
        imprimir_programa(prog[2], nivel+2)
        print(f"{indent}    Si Detectar es False:")
        imprimir_programa(prog[3], nivel+2)

if __name__ == "__main__":
    main()



--- Mejor Programa Encontrado ---
SECUENCIA (Ejecuta en orden):
    IF (Condicional):
        Si Detectar es True:
            IF (Condicional):
                Si Detectar es True:
                    - Entregar
                Si Detectar es False:
                    - S
        Si Detectar es False:
            SECUENCIA (Ejecuta en orden):
                IF (Condicional):
                    Si Detectar es True:
                        - N
                    Si Detectar es False:
                        - Detectar
                IF (Condicional):
                    Si Detectar es True:
                        - S
                    Si Detectar es False:
                        - N
    IF (Condicional):
        Si Detectar es True:
            IF (Condicional):
                Si Detectar es True:
                    - N
                Si Detectar es False:
                    - Entregar
        Si Detectar es False:
            IF (Condicional):
                Si Detectar 